<a href="https://colab.research.google.com/github/satishgaurav/MoLD/blob/main/1-molel_training_using_chemprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
from google.colab import userdata
# userdata.get('github')

In [2]:
!git clone https://github.com/satishgaurav/MoLD.git

Cloning into 'MoLD'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 162 (delta 23), reused 7 (delta 3), pack-reused 116
Receiving objects: 100% (162/162), 36.16 MiB | 27.72 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [3]:
# !git clone https://{userdata.get('github')}@github.com/satishgaurav/MoLD.git

In [4]:
!cp ./MoLD/data/*.csv ./

In [5]:
!ls -alh

total 5.4M
drwxr-xr-x 1 root root 4.0K Jun 21 15:34 .
drwxr-xr-x 1 root root 4.0K Jun 21 15:31 ..
-rw-r--r-- 1 root root 146K Jun 21 15:34 BBBP.csv
drwxr-xr-x 4 root root 4.0K Jun 18 13:23 .config
-rw-r--r-- 1 root root  61K Jun 21 15:34 fda_approved.csv
-rw-r--r-- 1 root root 2.1M Jun 21 15:34 HIV.csv
drwxr-xr-x 7 root root 4.0K Jun 21 15:34 MoLD
-rw-r--r-- 1 root root 3.1M Jun 21 15:34 named.csv
drwxr-xr-x 1 root root 4.0K Jun 18 13:23 sample_data
-rw-r--r-- 1 root root 2.8K Jun 21 15:34 substances.csv


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jun 21 15:34:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [8]:
!python --version

Python 3.10.12


In [9]:
!pip install chemprop
!pip install rdkit-pypi  # should be included in above after Chemprop v1.6 release

import chemprop
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.decomposition import PCA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 186.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 42.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusol

In [10]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
hiv_df = pd.read_csv("HIV.csv")
hiv_df.head()

,smiles,activity,HIV_active
0,CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...,CI,0
1,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...,CI,0
2,CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21,CI,0
3,Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1,CI,0
4,O=S(=O)(O)CCS(=O)(=O)O,CI,0


In [12]:
hiv_df.describe()

,HIV_active
count,41127.000000
mean,0.035086
std,0.184001
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [13]:
unique_values = hiv_df['HIV_active'].unique()
print(f"Unique values in 'HIV_active': {unique_values}")

Unique values in 'HIV_active': [0 1]


In [14]:
unique_values = hiv_df['activity'].unique()
print(f"Unique values in 'activity': {unique_values}")

Unique values in 'activity': ['CI' 'CM' 'CA']


In [15]:
unique_values = hiv_df['smiles'].unique()
print(f"Unique values in 'smiles': {unique_values}")
print(f"length of uniqe value: {len(unique_values)}")

Unique values in 'smiles': ['CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2'
 'C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3)CC(c3ccccc3)=[O+]2)[O+]=C(c2ccccc2)C1'
 'CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21' ...
 'Cc1ccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C)CC4C3C2=O)cc1'
 'Cc1cccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C)CC4C3C2=O)c1'
 'CCCCCC=C(c1cc(Cl)c(OC)c(-c2nc(C)no2)c1)c1cc(Cl)c(OC)c(-c2nc(C)no2)c1']
length of uniqe value: 41127


In [16]:
# Filter rows where 'your_column' is not equal to 1 or 0
filtered_df = hiv_df[(hiv_df['HIV_active'] != 1) & (hiv_df['HIV_active'] != 0)]
filtered_df

,smiles,activity,HIV_active


In [17]:
# Filter rows where 'target_column' is equal to 1h
hiv_df_filtered_active = hiv_df[hiv_df['HIV_active'] == 1]
hiv_df_filtered_active

,smiles,activity,HIV_active
11,O=C(O)Cc1ccc(SSc2ccc(CC(=O)O)cc2)cc1,CM,1
16,NNP(=S)(NN)c1ccccc1,CM,1
80,O=Nc1ccc(O)c(N=O)c1O,CM,1
203,Oc1ccc(Cl)cc1C(c1cc(Cl)ccc1O)C(Cl)(Cl)Cl,CM,1
234,NNC(=O)c1ccccc1SSc1ccccc1C(=O)NN,CM,1
...,...,...,...
41090,Cc1cn(COCCCOCC(=O)c2ccccc2)c(=O)[nH]c1=O,CM,1
41092,Cc1cn(C2CC3C(COC(CCC[Se]c4ccccc4)N3O)O2)c(=O)[...,CM,1
41093,Cc1cn(C2CC3C(COC(CCCC[Se]c4ccccc4)N3O)O2)c(=O)...,CM,1
41098,Cc1cn(C2CC3C(COC(CC[Se]C#N)N3O)O2)c(=O)[nH]c1=O,CM,1


In [18]:
# Filter rows where 'target_column' is equal to 1h
hiv_df_filtered_inactive = hiv_df[hiv_df['HIV_active'] == 0]
hiv_df_filtered_inactive = hiv_df_filtered_inactive.sample(n=1500, axis=0, random_state=42)
hiv_df_filtered_inactive

,smiles,activity,HIV_active
2428,O=C1c2ccccc2-c2nc3ccccc3nc21,CI,0
6197,O=C(CSc1cc(-c2ccc(Cl)cc2)s[s+]1)c1ccccc1,CI,0
17138,O=C(C=Nc1ccccc1C(=O)O)c1ccco1,CI,0
12261,CCCCCCCCCCCCCCCCCC[N+](C)(C)Cc1ccc(C[N+](C)(C)...,CI,0
3588,N#CSC1CCCCCCC1SC#N,CI,0
...,...,...,...
18477,CC(=O)OC1(C#N)CC2OC1C1C2N1C(=O)OC(C)(C)C,CI,0
1189,CCOC(=O)C1Cc2cc(C)c(C)cc2N(C)C1=O,CI,0
36657,CCOC(=O)N1CCN(c2ccc3c(C)cc(C)nc3n2)CC1,CI,0
27919,CN(C)C=Nc1ccc2c3c(cccc13)-c1ccccc1-2,CI,0


In [19]:
hiv_df_sampled = pd.concat([hiv_df_filtered_active, hiv_df_filtered_inactive], axis=0, ignore_index=True)
hiv_df_sampled

,smiles,activity,HIV_active
0,O=C(O)Cc1ccc(SSc2ccc(CC(=O)O)cc2)cc1,CM,1
1,NNP(=S)(NN)c1ccccc1,CM,1
2,O=Nc1ccc(O)c(N=O)c1O,CM,1
3,Oc1ccc(Cl)cc1C(c1cc(Cl)ccc1O)C(Cl)(Cl)Cl,CM,1
4,NNC(=O)c1ccccc1SSc1ccccc1C(=O)NN,CM,1
...,...,...,...
2938,CC(=O)OC1(C#N)CC2OC1C1C2N1C(=O)OC(C)(C)C,CI,0
2939,CCOC(=O)C1Cc2cc(C)c(C)cc2N(C)C1=O,CI,0
2940,CCOC(=O)N1CCN(c2ccc3c(C)cc(C)nc3n2)CC1,CI,0
2941,CN(C)C=Nc1ccc2c3c(cccc13)-c1ccccc1-2,CI,0


In [20]:
# Randomly shuffle rows
hiv_df_sampled = hiv_df_sampled.sample(frac=1, random_state=42)

In [21]:
hiv_df_sampled.head()
hiv_df.info()

,smiles,activity,HIV_active
840,C[N+](C)(C)C.O=C(Nc1ccc(C=Cc2ccc(NC(=O)c3cc(S(...,CM,1
1037,COC(=O)Cc1cc(O)c(CC=C(C)CCC=C(C)C(O)C(=O)C=C(C...,CM,1
2399,CCCCCCCCCCC#CCC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C...,CI,0
678,Cc1cc(C)c(S(=O)(O)=[OH+])c(C)c1.N[S+]1Cc2nc3cc...,CM,1
196,CCOc1cc(C2=NN3C(=S)NNC3=NN2)ccc1O,CM,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41127 entries, 0 to 41126
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   smiles      41127 non-null  object
 1   activity    41127 non-null  object
 2   HIV_active  41127 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 964.0+ KB


In [22]:
hiv_df_sampled.to_csv('HIV_2.csv', index=False)
# .drop(['activity'], axis=1).
hiv_df_sampled_2 = pd.read_csv("HIV_2.csv")
hiv_df_sampled_2.head()
hiv_df_sampled_2.tail()

,smiles,activity,HIV_active
0,C[N+](C)(C)C.O=C(Nc1ccc(C=Cc2ccc(NC(=O)c3cc(S(...,CM,1
1,COC(=O)Cc1cc(O)c(CC=C(C)CCC=C(C)C(O)C(=O)C=C(C...,CM,1
2,CCCCCCCCCCC#CCC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C...,CI,0
3,Cc1cc(C)c(S(=O)(O)=[OH+])c(C)c1.N[S+]1Cc2nc3cc...,CM,1
4,CCOc1cc(C2=NN3C(=S)NNC3=NN2)ccc1O,CM,1


,smiles,activity,HIV_active
2938,CCCCCCCCNC(=S)NC=C1C(=O)Oc2ccccc2C1=O,CI,0
2939,COP(=O)(C=Cc1cc(C(=O)OCc2ccccc2)n(S(=O)(=O)c2c...,CM,1
2940,O=C(CCc1ccc(O)cc1)NCCc1ccc(O)cc1,CM,1
2941,CCc1c(Cc2cc(C)cc(C)c2)n(COCCCO)c(=O)[nH]c1=O,CA,1
2942,CCSCCCCCCCCCCC(=O)OCC1OC(n2cc(C)c(=O)[nH]c2=O)...,CA,1


In [23]:
hiv_df_sampled_2

,smiles,activity,HIV_active
0,C[N+](C)(C)C.O=C(Nc1ccc(C=Cc2ccc(NC(=O)c3cc(S(...,CM,1
1,COC(=O)Cc1cc(O)c(CC=C(C)CCC=C(C)C(O)C(=O)C=C(C...,CM,1
2,CCCCCCCCCCC#CCC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C...,CI,0
3,Cc1cc(C)c(S(=O)(O)=[OH+])c(C)c1.N[S+]1Cc2nc3cc...,CM,1
4,CCOc1cc(C2=NN3C(=S)NNC3=NN2)ccc1O,CM,1
...,...,...,...
2938,CCCCCCCCNC(=S)NC=C1C(=O)Oc2ccccc2C1=O,CI,0
2939,COP(=O)(C=Cc1cc(C(=O)OCc2ccccc2)n(S(=O)(=O)c2c...,CM,1
2940,O=C(CCc1ccc(O)cc1)NCCc1ccc(O)cc1,CM,1
2941,CCc1c(Cc2cc(C)cc(C)c2)n(COCCCO)c(=O)[nH]c1=O,CA,1


In [24]:
arguments = [
    '--data_path', 'HIV_2.csv',
    '--dataset_type', 'classification',
    '--save_dir', 'test_checkpoints_multimolecule',
    '--epochs', '30',
    '--save_smiles_splits',
    '--quiet',
    '--batch_size', '64',
    '--ignore_columns', 'activity',
    '--depth', '5',
    # '--hidden_size', '1600',
    '--dropout', '0.2',
]

args = chemprop.args.TrainArgs().parse_args(arguments)

In [25]:
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

2943it [00:00, 80887.50it/s]
100%|██████████| 2943/2943 [00:02<00:00, 1449.11it/s]
Fold 0
2943it [00:00, 162759.41it/s]
100%|██████████| 37/37 [00:12<00:00,  5.50it/s]
                                               
 97%|█████████▋| 36/37 [00:03<00:00,  9.15it/s]
                                               
100%|██████████| 37/37 [00:04<00:00,  5.91it/s]
                                               
 97%|█████████▋| 36/37 [00:03<00:00,  9.26it/s]
                                               
 97%|█████████▋| 36/37 [00:03<00:00,  8.73it/s]
                                               
 97%|█████████▋| 36/37 [00:05<00:00,  9.10it/s]
                                               
100%|██████████| 37/37 [00:03<00:00, 10.15it/s]
                                               
100%|██████████| 37/37 [00:04<00:00,  3.59it/s]
                                               
100%|██████████| 37/37 [00:04<00:00,  9.16it/s]
                                               
 97%|█████████▋|

In [26]:
mean_score, std_score

(0.8445180889257111, 0.0)

In [27]:
bp_df = pd.read_csv("BBBP.csv")
bp_df.head()
bp_df.info()

,num,name,p_np,smiles
0,1,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,2,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,3,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,4,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,5,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   num     2050 non-null   int64 
 1   name    2050 non-null   object
 2   p_np    2050 non-null   int64 
 3   smiles  2050 non-null   object
dtypes: int64(2), object(2)
memory usage: 64.2+ KB


In [28]:
bp_df.tail()

,num,name,p_np,smiles
2045,2049,licostinel,1,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl
2046,2050,ademetionine(adenosyl-methionine),1,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...
2047,2051,mesocarb,1,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...
2048,2052,tofisoline,1,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...
2049,2053,azidamfenicol,1,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...


In [29]:
bp_df.drop(['num', 'name', 'p_np'], axis=1).to_csv('BBBP_2.csv', index=False)

In [30]:
bp_df_2 = pd.read_csv("BBBP_2.csv")
bp_df_2.tail()
bp_df_2.info()

,smiles
2045,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl
2046,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...
2047,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...
2048,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...
2049,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   smiles  2050 non-null   object
dtypes: object(1)
memory usage: 16.1+ KB


In [31]:
arguments = [
    '--test_path', 'BBBP_2.csv',
    '--preds_path', 'BBBP_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


2050it [00:00, 212419.67it/s]
100%|██████████| 2050/2050 [00:00<00:00, 125195.81it/s]

Validating SMILES



[15:38:37] Explicit valence for atom # 1 N, 4, is greater than permitted
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] Explicit valence for atom # 6 N, 4, is greater than permitted
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] Explicit valence for atom # 6 N, 4, is greater than permitted
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] WARNING: not removing hydrogen atom without neighbors
[15:38:37] Explicit valence for atom # 11 N, 4, is greater than p

Test size = 2,039


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [00:08<00:00,  8.78s/it]

Saving predictions to BBBP_preds.csv
Elapsed time = 0:00:10


In [32]:
bp_preds_df = pd.read_csv("BBBP_preds.csv")
bp_preds_df.head()

,smiles,HIV_active
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,0.08995568752288818
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,0.13612912595272064
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,0.2891809046268463
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,0.07175479829311371
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,0.3540500998497009


In [33]:
bp_preds_df.tail()

,smiles,HIV_active
2045,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl,0.28399333357810974
2046,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...,0.13631369173526764
2047,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...,0.4787648916244507
2048,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...,0.33214083313941956
2049,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...,0.479694128036499


In [34]:
bp_preds_df.describe()

,smiles,HIV_active
count,2050,2050
unique,2050,1997
top,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,Invalid SMILES
freq,1,11


In [35]:
bp_preds_df = bp_preds_df[bp_preds_df['HIV_active'] != "Invalid SMILES"]
bp_preds_df.describe()

,smiles,HIV_active
count,2039,2039
unique,2039,1996
top,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,0.06751451641321182
freq,1,3


In [36]:
bp_preds_df['HIV_active'] = bp_preds_df['HIV_active'].astype(float)

In [37]:
bp_preds_df['HIV_active_2'] = bp_preds_df['HIV_active'].apply(lambda x: 1 if x > 0.9 else 0)
bp_preds_df

,smiles,HIV_active,HIV_active_2
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,0.089956,0
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,0.136129,0
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,0.289181,0
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,0.071755,0
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,0.354050,0
...,...,...,...
2045,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl,0.283993,0
2046,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...,0.136314,0
2047,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...,0.478765,0
2048,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...,0.332141,0


In [38]:
bp_preds_df.describe()

,HIV_active,HIV_active_2
count,2039.000000,2039.000000
mean,0.294235,0.008337
std,0.234286,0.090950
min,0.000125,0.000000
25%,0.101336,0.000000
50%,0.214237,0.000000
75%,0.452856,0.000000
max,0.999444,1.000000


In [39]:
# Filter rows where 'target_column' is equal to 1
bp_preds_df_filtered = bp_preds_df[bp_preds_df['HIV_active_2'] == 1]
bp_preds_df_filtered


,smiles,HIV_active,HIV_active_2
11,CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O,0.901065,1
319,CC1=CN([C@@H]2O[C@H](CO)C=C2)C(=O)NC1=O,0.955664,1
427,CC(C)[C@@H]1NC(=O)[C@H](C)OC(=O)C(NC(=O)[C@H](...,0.951809,1
445,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...,0.991337,1
527,CN(C)CC\C=C/1c2ccccc2Sc3ccc(Cl)cc13,0.951663,1
912,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(NC(=...,0.939820,1
945,Oc1ccc(cc1)/C=C([N+]#[C-])/C(=C/c2ccc(O)cc2)[N...,0.956967,1
973,COC1(C(F)(F)C1(F)Cl)F,0.986301,1
1004,C(C1(C(NC(=O)NC1=O)=O)C(C)C)C=C,0.940815,1
1165,CC1(C(NC(O1)=O)=O)C,0.903444,1


In [40]:
smiles_to_check = bp_preds_df_filtered['smiles'].to_list()

In [41]:
hiv_df_sampled_2[hiv_df_sampled_2['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [42]:
hiv_df[hiv_df['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [43]:
bp_df[bp_df['smiles'].isin(smiles_to_check)]

,num,name,p_np,smiles
11,12,alovudine,1,CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O
319,321,Stavudine,1,CC1=CN([C@@H]2O[C@H](CO)C=C2)C(=O)NC1=O
427,429,valinomycin,0,CC(C)[C@@H]1NC(=O)[C@H](C)OC(=O)C(NC(=O)[C@H](...
445,447,zidovudine,0,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...
527,529,chlorprothixene,1,CN(C)CC\C=C/1c2ccccc2Sc3ccc(Cl)cc13
912,914,rifamycin,0,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(NC(=...
945,947,xantocillin,0,Oc1ccc(cc1)/C=C([N+]#[C-])/C(=C/c2ccc(O)cc2)[N...
973,975,aliflurane,1,COC1(C(F)(F)C1(F)Cl)F
1004,1006,aprobarbital,1,C(C1(C(NC(=O)NC1=O)=O)C(C)C)C=C
1165,1169,dimethadione,1,CC1(C(NC(O1)=O)=O)C


In [44]:
bp_df_final = pd.merge(bp_df[bp_df['smiles'].isin(smiles_to_check)], bp_preds_df_filtered, on='smiles' )
bp_df_final

,num,name,p_np,smiles,HIV_active,HIV_active_2
0,12,alovudine,1,CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O,0.901065,1
1,321,Stavudine,1,CC1=CN([C@@H]2O[C@H](CO)C=C2)C(=O)NC1=O,0.955664,1
2,429,valinomycin,0,CC(C)[C@@H]1NC(=O)[C@H](C)OC(=O)C(NC(=O)[C@H](...,0.951809,1
3,447,zidovudine,0,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...,0.991337,1
4,529,chlorprothixene,1,CN(C)CC\C=C/1c2ccccc2Sc3ccc(Cl)cc13,0.951663,1
5,914,rifamycin,0,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(NC(=...,0.939820,1
6,947,xantocillin,0,Oc1ccc(cc1)/C=C([N+]#[C-])/C(=C/c2ccc(O)cc2)[N...,0.956967,1
7,975,aliflurane,1,COC1(C(F)(F)C1(F)Cl)F,0.986301,1
8,1006,aprobarbital,1,C(C1(C(NC(=O)NC1=O)=O)C(C)C)C=C,0.940815,1
9,1169,dimethadione,1,CC1(C(NC(O1)=O)=O)C,0.903444,1


In [45]:
bp_df_final.to_csv('HIV_result.csv', index=False)

In [46]:
sub_df = pd.read_csv("substances.csv")
sub_df.head()

,zinc_id,smiles
0,ZINC000000000027,N[C@@H](CCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
1,ZINC000016090786,N[C@H](CCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
2,ZINC000001763088,N[C@H](CCCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
3,ZINC000002033385,N[C@@H](CCCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
4,ZINC000000001673,N[C@@H](Cc1ccc(N(CCCl)CCCl)cc1)C(=O)O


In [47]:
sub_df.tail()

,zinc_id,smiles
46,ZINC000196349655,O=C(O)CCSc1ccc(N(CCCl)CCCl)cc1
47,ZINC000064454242,N=NCCCc1ccc(N(CCCl)CCCl)cc1
48,ZINC000005161807,O=C(O)C/C=C/c1ccc(N(CCCl)CCCl)cc1
49,ZINC000001682294,O=C(O)CCOc1ccc(N(CCCl)CCCl)cc1
50,ZINC000079564304,O=C(O)CNC(=O)c1ccc(N(CCCl)CCCl)cc1


In [48]:
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   zinc_id  51 non-null     object
 1   smiles   51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


In [49]:
arguments = [
    '--test_path', 'substances.csv',
    '--preds_path', 'substances_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule',
    '--smiles_columns', 'smiles'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


51it [00:00, 32430.19it/s]
100%|██████████| 51/51 [00:00<00:00, 75135.06it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validating SMILES
Test size = 51


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [00:00<00:00,  1.38it/s]

Saving predictions to substances_preds.csv
Elapsed time = 0:00:01


In [50]:
fda_df = pd.read_csv("fda_approved.csv")
fda_df.head()

,zinc_id,smiles
0,ZINC000001530427,C[C@@H]1O[C@@H]1P(=O)(O)O
1,ZINC000003807804,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1
2,ZINC000000120286,Nc1nc(N)c2nc(-c3ccccc3)c(N)nc2n1
3,ZINC000242548690,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...
4,ZINC000000008492,Oc1cccc2cccnc12


In [51]:
arguments = [
    '--test_path', 'fda_approved.csv',
    '--preds_path', 'fda_approved_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule',
    '--smiles_columns', 'smiles'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


892it [00:00, 165993.13it/s]
100%|██████████| 892/892 [00:00<00:00, 125052.45it/s]

Validating SMILES



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test size = 892


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [00:02<00:00,  2.97s/it]

Saving predictions to fda_approved_preds.csv
Elapsed time = 0:00:04


In [52]:
fda_preds_df = pd.read_csv("fda_approved_preds.csv")
fda_preds_df.head()
fda_preds_df.info()

,zinc_id,smiles,HIV_active
0,ZINC000001530427,C[C@@H]1O[C@@H]1P(=O)(O)O,0.148917
1,ZINC000003807804,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.498898
2,ZINC000000120286,Nc1nc(N)c2nc(-c3ccccc3)c(N)nc2n1,0.120655
3,ZINC000242548690,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,0.707267
4,ZINC000000008492,Oc1cccc2cccnc12,0.122415


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   zinc_id     892 non-null    object 
 1   smiles      892 non-null    object 
 2   HIV_active  892 non-null    float64
dtypes: float64(1), object(2)
memory usage: 21.0+ KB


In [53]:
fda_preds_df = fda_preds_df[fda_preds_df['HIV_active'] != "Invalid SMILES"]
fda_preds_df.describe()
fda_preds_df['HIV_active'] = fda_preds_df['HIV_active'].astype(float)
fda_preds_df['HIV_active_2'] = fda_preds_df['HIV_active'].apply(lambda x: 1 if x > 0.9 else 0)
fda_preds_df.head()

,HIV_active
count,892.000000
mean,0.279337
std,0.225053
min,0.001319
25%,0.104242
50%,0.194162
75%,0.402606
max,0.991847


,zinc_id,smiles,HIV_active,HIV_active_2
0,ZINC000001530427,C[C@@H]1O[C@@H]1P(=O)(O)O,0.148917,0
1,ZINC000003807804,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.498898,0
2,ZINC000000120286,Nc1nc(N)c2nc(-c3ccccc3)c(N)nc2n1,0.120655,0
3,ZINC000242548690,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,0.707267,0
4,ZINC000000008492,Oc1cccc2cccnc12,0.122415,0


In [54]:
# Filter rows where 'target_column' is equal to 1
fda_preds_df_filtered = fda_preds_df[fda_preds_df['HIV_active_2'] == 1]
fda_preds_df_filtered

,zinc_id,smiles,HIV_active,HIV_active_2
94,ZINC000068153186,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...,0.966764,1
340,ZINC000001530621,CCN[C@H]1C[C@H](C)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,0.903742,1
433,ZINC000003831531,Cc1ccsc1C(=CCCN1CCC[C@@H](C(=O)O)C1)c1sccc1C,0.921090,1
540,ZINC000004474564,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)O,0.907606,1
553,ZINC000100032379,N=C1CCCN1Cc1[nH]c(=O)[nH]c(=O)c1Cl,0.914277,1
648,ZINC000001530636,N=C(N)Nc1nc(CSCCC(=N)NS(N)(=O)=O)cs1,0.920087,1
715,ZINC000169289767,Cc1cc(-c2ccc(/N=N/c3c(S(=O)(=O)O)cc4cc(S(=O)(=...,0.991847,1
717,ZINC000169621223,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(c(/C...,0.914670,1
727,ZINC000003807172,C[C@H]1CNc2c(cccc2S(=O)(=O)N[C@@H](CCCNC(=N)N)...,0.948526,1
819,ZINC000000137884,Cc1cn([C@H]2C=C[C@@H](CO)O2)c(=O)[nH]c1=O,0.943154,1


In [55]:
smiles_to_check = fda_preds_df_filtered['smiles'].to_list()
print(f"smiles to check: {smiles_to_check}")

smiles to check: ['CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2F)c(-c2ccnc(N)n2)s1', 'CCN[C@H]1C[C@H](C)S(=O)(=O)c2sc(S(N)(=O)=O)cc21', 'Cc1ccsc1C(=CCCN1CCC[C@@H](C(=O)O)C1)c1sccc1C', 'CC/C=C\\C/C=C\\C/C=C\\C/C=C\\C/C=C\\C/C=C\\CCC(=O)O', 'N=C1CCCN1Cc1[nH]c(=O)[nH]c(=O)c1Cl', 'N=C(N)Nc1nc(CSCCC(=N)NS(N)(=O)=O)cs1', 'Cc1cc(-c2ccc(/N=N/c3c(S(=O)(=O)O)cc4cc(S(=O)(=O)O)cc(N)c4c3O)c(C)c2)ccc1/N=N/c1c(S(=O)(=O)O)cc2cc(S(=O)(=O)O)cc(N)c2c1O', 'CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(c(/C=N/N5CCN(C)CC5)c(O)c4c3C2=O)NC(=O)/C(C)=C\\C=C\\[C@H](C)[C@H](O)[C@@H](C)[C@@H](O)[C@@H](C)[C@H](OC(C)=O)[C@@H]1C', 'C[C@H]1CNc2c(cccc2S(=O)(=O)N[C@@H](CCCNC(=N)N)C(=O)N2CC[C@@H](C)C[C@@H]2C(=O)O)C1', 'Cc1cn([C@H]2C=C[C@@H](CO)O2)c(=O)[nH]c1=O', 'CC1(C)[C@H](C(=O)O)N2C(=O)C[C@H]2S1(=O)=O', 'Cc1cn([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)c(=O)[nH]c1=O']


In [56]:
hiv_df_sampled_2[hiv_df_sampled_2['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [57]:
hiv_df[hiv_df['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [58]:
bp_df[bp_df['smiles'].isin(smiles_to_check)]

,num,name,p_np,smiles


In [59]:
fda_df[fda_df['smiles'].isin(smiles_to_check)]

,zinc_id,smiles
94,ZINC000068153186,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...
340,ZINC000001530621,CCN[C@H]1C[C@H](C)S(=O)(=O)c2sc(S(N)(=O)=O)cc21
433,ZINC000003831531,Cc1ccsc1C(=CCCN1CCC[C@@H](C(=O)O)C1)c1sccc1C
540,ZINC000004474564,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)O
553,ZINC000100032379,N=C1CCCN1Cc1[nH]c(=O)[nH]c(=O)c1Cl
648,ZINC000001530636,N=C(N)Nc1nc(CSCCC(=N)NS(N)(=O)=O)cs1
715,ZINC000169289767,Cc1cc(-c2ccc(/N=N/c3c(S(=O)(=O)O)cc4cc(S(=O)(=...
717,ZINC000169621223,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(c(/C...
727,ZINC000003807172,C[C@H]1CNc2c(cccc2S(=O)(=O)N[C@@H](CCCNC(=N)N)...
819,ZINC000000137884,Cc1cn([C@H]2C=C[C@@H](CO)O2)c(=O)[nH]c1=O


In [60]:
fda_df_final = pd.merge(fda_df[fda_df['smiles'].isin(smiles_to_check)], fda_preds_df_filtered, on='smiles' )
fda_df_final

,zinc_id_x,smiles,zinc_id_y,HIV_active,HIV_active_2
0,ZINC000068153186,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...,ZINC000068153186,0.966764,1
1,ZINC000001530621,CCN[C@H]1C[C@H](C)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,ZINC000001530621,0.903742,1
2,ZINC000003831531,Cc1ccsc1C(=CCCN1CCC[C@@H](C(=O)O)C1)c1sccc1C,ZINC000003831531,0.921090,1
3,ZINC000004474564,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)O,ZINC000004474564,0.907606,1
4,ZINC000100032379,N=C1CCCN1Cc1[nH]c(=O)[nH]c(=O)c1Cl,ZINC000100032379,0.914277,1
5,ZINC000001530636,N=C(N)Nc1nc(CSCCC(=N)NS(N)(=O)=O)cs1,ZINC000001530636,0.920087,1
6,ZINC000169289767,Cc1cc(-c2ccc(/N=N/c3c(S(=O)(=O)O)cc4cc(S(=O)(=...,ZINC000169289767,0.991847,1
7,ZINC000169621223,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(c(/C...,ZINC000169621223,0.914670,1
8,ZINC000003807172,C[C@H]1CNc2c(cccc2S(=O)(=O)N[C@@H](CCCNC(=N)N)...,ZINC000003807172,0.948526,1
9,ZINC000000137884,Cc1cn([C@H]2C=C[C@@H](CO)O2)c(=O)[nH]c1=O,ZINC000000137884,0.943154,1


In [61]:
fda_df_final.to_csv('fda_approved_result.csv', index=False)

In [62]:
# !wget https://zinc15.docking.org/substances/subsets/named.csv

In [63]:
zinc_df = pd.read_csv("named.csv")
zinc_df.head()
zinc_df.tail()
zinc_df.info()

,zinc_id,smiles
0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...
1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...
2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...
3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl
4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...


,zinc_id,smiles
34595,ZINC000005999135,COc1cc([C@@H]2Oc3c(OC)cc(/C=C/CO)cc3[C@H]2CO)c...
34596,ZINC000084710404,COC(=O)c1cc(OC)c2cc(OC)c(OC)c(O)c2c1O
34597,ZINC000150369761,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)O[...
34598,ZINC000095098911,CCCCCCCCCCCC[C@H](O)[C@H]1CC[C@H]([C@H](O)CCCC...
34599,ZINC000000001009,C[N+](C)([O-])CC/C=C1\c2ccccc2C=Cc2c(Cl)cccc21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34600 entries, 0 to 34599
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   zinc_id  34600 non-null  object
 1   smiles   34600 non-null  object
dtypes: object(2)
memory usage: 540.8+ KB


In [64]:
arguments = [
    '--test_path', 'named.csv',
    '--preds_path', 'named_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule',
    '--smiles_columns', 'smiles'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


34600it [00:00, 227815.33it/s]
100%|██████████| 34600/34600 [00:00<00:00, 48244.11it/s]


Validating SMILES


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test size = 34,600


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [03:12<00:00, 192.43s/it]


Saving predictions to named_preds.csv
Elapsed time = 0:03:26


In [65]:
zinc_preds_df = pd.read_csv("named_preds.csv")
zinc_preds_df.head()
zinc_preds_df = zinc_preds_df[zinc_preds_df['HIV_active'] != "Invalid SMILES"]
zinc_preds_df.describe()
zinc_preds_df['HIV_active'] = zinc_preds_df['HIV_active'].astype(float)
zinc_preds_df['HIV_active_2'] = zinc_preds_df['HIV_active'].apply(lambda x: 1 if x > 0.9 else 0)
zinc_preds_df.head()

,zinc_id,smiles,HIV_active
0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...,0.558561
1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...,0.675089
2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...,0.342106
3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl,0.085523
4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...,0.484810


,HIV_active
count,34600.000000
mean,0.521786
std,0.269715
min,0.000005
25%,0.281168
50%,0.561346
75%,0.768974
max,0.999807


,zinc_id,smiles,HIV_active,HIV_active_2
0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...,0.558561,0
1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...,0.675089,0
2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...,0.342106,0
3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl,0.085523,0
4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...,0.484810,0


In [66]:
# Filter rows where 'target_column' is equal to 1
zinc_preds_df_filtered = zinc_preds_df[zinc_preds_df['HIV_active_2'] == 1]
zinc_preds_df_filtered

,zinc_id,smiles,HIV_active,HIV_active_2
21,ZINC000040753343,COc1cc(/C=C/c2cc(O)c(CC=C(C)C)c(O)c2)cc2c1O[C@...,0.956375,1
124,ZINC000150343906,CCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](CO[P@@](=O...,0.907218,1
128,ZINC000008220075,Nc1nc(O)c(N)c(N[C@@H]2O[C@H](CO[P@@](=O)(O)O[P...,0.981322,1
138,ZINC000096888407,CC(C)[C@H]1C=C[C@]2(C)Oc3c(C(=O)/C=C/c4ccccc4)...,0.905815,1
149,ZINC000014680924,Cc1c(Cl)c(O)cc(O)c1C(=O)O[C@H]1C[C@]2(C)[C@@H]...,0.968062,1
...,...,...,...,...
34482,ZINC000014650056,CC(C)=CCC[C@@]1(C)C=Cc2c(-c3oc4cc(O)cc(O)c4c(=...,0.933316,1
34489,ZINC000014647364,COc1cc(-c2[o+]c3cc(O)cc(O)c3cc2O[C@@H]2O[C@H](...,0.914603,1
34565,ZINC000027655377,CC1=C[C@H]2O[C@@H]3[C@H](O)[C@@H](O)[C@@](C)([...,0.922007,1
34571,ZINC000013414448,COc1cc(O)c2c3c1OC(C)(C)[C@H]3Cc1c-2oc2c3c(cc(O...,0.913769,1


In [67]:
zinc_preds_df_filtered.to_csv('zinc_final_result.csv', index=False)

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [70]:
!mkdir '/content/drive/My Drive/Chemprop_Backup_5/'

In [71]:
!pwd

/content


In [72]:
!ls -al

total 9868
drwxr-xr-x 1 root root    4096 Jun 21 15:47 .
drwxr-xr-x 1 root root    4096 Jun 21 15:31 ..
-rw-r--r-- 1 root root  107579 Jun 21 15:38 BBBP_2.csv
-rw-r--r-- 1 root root  148743 Jun 21 15:34 BBBP.csv
-rw-r--r-- 1 root root  149726 Jun 21 15:38 BBBP_preds.csv
drwxr-xr-x 4 root root    4096 Jun 18 13:23 .config
drwx------ 5 root root    4096 Jun 21 15:47 drive
-rw-r--r-- 1 root root   61865 Jun 21 15:34 fda_approved.csv
-rw-r--r-- 1 root root   79339 Jun 21 15:38 fda_approved_preds.csv
-rw-r--r-- 1 root root    1490 Jun 21 15:38 fda_approved_result.csv
-rw-r--r-- 1 root root  174576 Jun 21 15:35 HIV_2.csv
-rw-r--r-- 1 root root 2193844 Jun 21 15:34 HIV.csv
-rw-r--r-- 1 root root    1611 Jun 21 15:38 HIV_result.csv
drwxr-xr-x 7 root root    4096 Jun 21 15:34 MoLD
-rw-r--r-- 1 root root 3160219 Jun 21 15:34 named.csv
-rw-r--r-- 1 root root 3824723 Jun 21 15:42 named_preds.csv
drwxr-xr-x 1 root root    4096 Jun 18 13:23 sample_data
-rw-r--r-- 1 root root    2767 Jun 21 15:34 sub

In [73]:
!cp -r /content/*.csv "/content/drive/My Drive/Chemprop_Backup_5/"

In [74]:
!cp -r /content/test_checkpoints_multimolecule/ "/content/drive/My Drive/Chemprop_Backup_5/"